## Install package and download required data files

In [3]:
# Download ADSO ontology file
!wget "https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl"

--2023-01-30 04:47:32--  https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68571 (67K) [text/plain]
Saving to: ‘ADSOv1.0.3.owl.1’

ADSOv1.0.3.owl.1    100%[===================>]  66.96K  --.-KB/s    in 0.002s  

2023-01-30 04:47:32 (38.2 MB/s) - ‘ADSOv1.0.3.owl.1’ saved [68571/68571]



In [4]:
!pip install rdflib spacy sentence-transformers huggingface-hub transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load required pre-trained models




In [5]:
# Download Wordnet dependencies
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords

# Get a list of English stop words
stop_words = stopwords.words('english')

In [7]:
from sentence_transformers import SentenceTransformer, util
from huggingface_hub import from_pretrained_keras
import numpy as np
import transformers
import tensorflow as tf
import joblib

# POS tagger
pos_tagger = joblib.load("/content/pos-tagger.joblib")

# Text similarity model
tsm = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2');

# Semantic similarity model
ssm = from_pretrained_keras("keras-io/bert-semantic-similarity")

config.json not found in HuggingFace Hub


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
# Helper functions to define the properties of the pos-tagger model
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

## Helper Utils

This block contains functions/classes/enums that use throught out the entity disambiguation component.

In [9]:
# Help utils for semantic similarity model
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

In [10]:
labels = ["negative_similarity", "positive_similarity", "neutral"]

## Data loader component

In here we are going to retreive the all type of entities/nodes from the ontology knowledgebase and store in seperate arrays.



In [11]:
# Load downloaded ontology graph
from rdflib import Graph, Namespace, Literal, RDF, URIRef

g = Graph()
g.parse("/content/ADSOv1.0.3.owl")

<Graph identifier=N39af3b12f00542aa8908ea8424a9c617 (<class 'rdflib.graph.Graph'>)>

In [12]:
# Get the URIRef for given keyword
def get_ref(keyword):
    return URIRef("https://ontology.drpawspaw.com/"+keyword)

In [13]:
# Get the entity 'text' from the URIRef
def get_text_from_uri(uri):
    for s,p,o in g:
        if s == uri and p == URIRef("https://ontology.drpawspaw.com/text"):
            return o
    return None

In [14]:
print(get_text_from_uri(URIRef('https://ontology.drpawspaw.com/ADSO0000000083')))

Distempter


In [61]:
# Get list of symptoms from disease URI
def get_symptoms_from_disease_uri(uri):
  symptoms = []
  for s,p,o in g:
    if s == uri and p == URIRef("https://ontology.drpawspaw.com/hasSymptom"):
      symptoms.append(o.toPython())
  return symptoms

In [62]:
get_symptoms_from_disease_uri(URIRef('https://ontology.drpawspaw.com/ADSO0000000083'))

['https://ontology.drpawspaw.com/ADSO0000000027',
 'https://ontology.drpawspaw.com/ADSO0000000034',
 'https://ontology.drpawspaw.com/ADSO0000000031',
 'https://ontology.drpawspaw.com/ADSO0000000033',
 'https://ontology.drpawspaw.com/ADSO0000000036',
 'https://ontology.drpawspaw.com/ADSO0000000030',
 'https://ontology.drpawspaw.com/ADSO0000000040',
 'https://ontology.drpawspaw.com/ADSO0000000026',
 'https://ontology.drpawspaw.com/ADSO0000000017',
 'https://ontology.drpawspaw.com/ADSO0000000037',
 'https://ontology.drpawspaw.com/ADSO0000000032',
 'https://ontology.drpawspaw.com/ADSO0000000005',
 'https://ontology.drpawspaw.com/ADSO0000000038',
 'https://ontology.drpawspaw.com/ADSO0000000023',
 'https://ontology.drpawspaw.com/ADSO0000000039',
 'https://ontology.drpawspaw.com/ADSO0000000008',
 'https://ontology.drpawspaw.com/ADSO0000000093',
 'https://ontology.drpawspaw.com/ADSO0000000009',
 'https://ontology.drpawspaw.com/ADSO0000000007',
 'https://ontology.drpawspaw.com/ADSO0000000028']

In [17]:
# Get URI from text
def get_uri_from_text(text):
  for s, p, o in g:
    if p == URIRef("https://ontology.drpawspaw.com/text"):
      if o.toPython().lower() == text.lower():
        return s
  return None

In [18]:
get_uri_from_text("fever")

rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000005')

In [19]:
# Collect named entities from ontology
diseases = []
symptoms = []
synonyms = []

for s,p,o in g:
    if p == get_ref("hasDisease"):
        try:
            dis = get_text_from_uri(o).toPython()
            diseases.append(dis)
        except:
            print(type(o), o)
    if p == get_ref("hasSymptom"):
        try:
            sym = get_text_from_uri(o).toPython()
            symptoms.append(sym)
        except:
            print(type(o), o)
    if p == get_ref("hasSynonym"):
        try:
            syn = get_text_from_uri(o).toPython()
            synonyms.append(syn)
        except:
            print(type(o), o)

<class 'rdflib.term.URIRef'> https://ontology.drpawspaw.com/ADSO0000000110


In [20]:
# List of symptoms and their synonyms
symp_syn = []

# Check the keyword already exist or not in collection
def is_exist_ss(keyword):
    for _,k,_ in symp_syn:
        if k == keyword:
            return True
            
# Apped all the symptoms and their synonyms
for s,p,o in g: # subject, predicate, object
    # filter synonyms
    if p == get_ref("hasSymptom"):
        try:
            x = get_text_from_uri(o).toPython()
        except:
            continue
        curr_symp_syn = []
        for s1, p1, o1 in g:
            # filter synonym for above "o" entity
            if s1 == o and p1 == get_ref("hasSynonym"):
                try:
                    y = get_text_from_uri(o1).toPython()
                    curr_symp_syn.append(y)                    
                except:
                    continue
        # validate to add only one entry
        if not is_exist_ss(x):
            try:
                idx = o.toPython()
                symp_syn.append((idx, x, curr_symp_syn))
            except:
                continue

In [21]:
symp_syn[:5]

[('https://ontology.drpawspaw.com/ADSO0000000027', 'Eye Discharge', []),
 ('https://ontology.drpawspaw.com/ADSO0000000034', 'Muscle Twitching', []),
 ('https://ontology.drpawspaw.com/ADSO0000000016', 'Blood Diarrhea', []),
 ('https://ontology.drpawspaw.com/ADSO0000000029',
  'Scratching Ears',
  ['Crusting in the Ears']),
 ('https://ontology.drpawspaw.com/ADSO0000000042', 'Running Nose', [])]

In [22]:
len(symp_syn)

79

## Text similarity

In here we are going to calculate the similarity of the words using pretrained model from Hugging Face => Text Similarity - To get the similar words for identified named entities (https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2, https://huggingface.co/tasks/sentence-similarity)

In [23]:
# Generate synonyms for the existing symptoms and update the list of synonyms
from nltk.corpus import wordnet

def generate_synonyms(uri, keyword, syns):
    for synonym in wordnet.synsets(keyword):
        for item in synonym.lemmas():
            if keyword != synonym.name() and len(synonym.lemma_names()) > 1:
                syns.append(item.name())
    return (uri, keyword, syns)

In [24]:
# for idx,(uri,keyword,syns) in enumerate(symp_syn):
#   symp_syn[idx] = generate_synonyms(uri,keyword,syns)

In [25]:
print(len(symp_syn))
print(symp_syn[:5])

79
[('https://ontology.drpawspaw.com/ADSO0000000027', 'Eye Discharge', []), ('https://ontology.drpawspaw.com/ADSO0000000034', 'Muscle Twitching', []), ('https://ontology.drpawspaw.com/ADSO0000000016', 'Blood Diarrhea', []), ('https://ontology.drpawspaw.com/ADSO0000000029', 'Scratching Ears', ['Crusting in the Ears']), ('https://ontology.drpawspaw.com/ADSO0000000042', 'Running Nose', [])]


In [26]:
# Exapand the all entities
expanded_symp = []

def is_exist_es(keyword):
    for idx,word in expanded_symp:
        if keyword == word:
            return True

for idx,word,syns in symp_syn:
    for s in syns:
        if not is_exist_es(s):
            expanded_symp.append((idx, s))
    if not is_exist_es(word):
        expanded_symp.append((idx, word))

In [27]:
print(len(expanded_symp))
print(expanded_symp[:5])

101
[('https://ontology.drpawspaw.com/ADSO0000000027', 'Eye Discharge'), ('https://ontology.drpawspaw.com/ADSO0000000034', 'Muscle Twitching'), ('https://ontology.drpawspaw.com/ADSO0000000016', 'Blood Diarrhea'), ('https://ontology.drpawspaw.com/ADSO0000000029', 'Crusting in the Ears'), ('https://ontology.drpawspaw.com/ADSO0000000029', 'Scratching Ears')]


In [28]:
# Calculate the similariity
# Here "tsm" means the text-similarity-model, that we loaded in "Load required pre-trained models"
# "get_similarity" accepts the URI, identified named entity, entity from ontology
def get_similarity(idx, word, ne): # Return a tuple, contain URI, node, similarity_score
    embedding_1 = tsm.encode(ne, convert_to_tensor=True)
    embedding_2 = tsm.encode(word, convert_to_tensor=True)
    return (idx, word, util.pytorch_cos_sim(embedding_1, embedding_2))

In [29]:
# At the current implementation, we only get first five entities
# "entity" - identified named entity, "nodes" - named entities in the onotology (in format of [(idx, text)])
def get_most_similarity_entities(entity, nodes):
  scores = map(lambda e: get_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=lambda x:x[2], reverse=True)[:5]

In [30]:
# Tryout the implementation
identified_ne = ["fever", "lethargy"]

# Get the top five entities from the text similarity, accroding to the cosine similarity score
get_most_similarity_entities(identified_ne[0], expanded_symp)

[('https://ontology.drpawspaw.com/ADSO0000000005',
  'Fever',
  tensor([[1.]], device='cuda:0')),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Fever',
  tensor([[0.8284]], device='cuda:0')),
 ('https://ontology.drpawspaw.com/ADSO0000000032',
  'Pneumonia',
  tensor([[0.4948]], device='cuda:0')),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Temperature',
  tensor([[0.4413]], device='cuda:0')),
 ('https://ontology.drpawspaw.com/ADSO0000000008',
  'Vomiting',
  tensor([[0.4372]], device='cuda:0'))]

## Semantic Similarity

In here we are going to calculate the semantic similarity of the words using pretrained model from HuggingFace => Semantic Similarity - To get the similarity in context for the ranked candidates (https://huggingface.co/keras-io/bert-semantic-similarity)

In [31]:
# Calculate the semantic similarity of the identified entity and nodes in ontology
# "calculate_semantic_similarity" functions accpets, the URI, text and identified entity
def calculate_semantic_similarity(uri, ctx, entity):
    sentence_pairs = np.array([[str(entity), str(ctx)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = ssm.predict(test_data[0])[0]
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return (uri, ctx, labels_probs["positive_similarity"])

In [32]:
# "get_highest_ctx_similarity" function return the highest context similarity as tuple (URI, Word, ctx_similarity)
def get_highest_ctx_similarity(entity, nodes):
  scores = map(lambda e: calculate_semantic_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=lambda x:x[2], reverse=True)[:1]

In [33]:
ine = "fever"
candidates = get_most_similarity_entities(ine, expanded_symp) # Candidates from text-similarity
get_highest_ctx_similarity(ine, candidates)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 2s 2s/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 71ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 60ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 61ms/step


[('https://ontology.drpawspaw.com/ADSO0000000005',
  'Fever',
  0.9982507824897766)]

## Query data

In [34]:
# This function return SPARQL query, that able to get the disease from symptoms
def build_query(symptoms):
  return """
    PREFIX adso: <https://ontology.drpawspaw.com/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?diseaseName
    WHERE {{
        {sym_query}
        ?diseaseUri adso:text ?diseaseName .
    }}
    """.format(sym_query="\n".join(
        list(
            map(lambda e: "?diseaseUri adso:hasSymptom adso:{uri} .".format(uri=e),
                list(map(lambda e: e.split("/")[3], symptoms))))))

In [35]:
q = build_query(["https://ontology.drpawspaw.com/ADSO0000000005",
    "https://ontology.drpawspaw.com/ADSO0000000006",
    "https://ontology.drpawspaw.com/ADSO0000000007"])

for output in g.query(q):
  print(output)

(rdflib.term.Literal('Babesiosis', lang='en'),)


## Implementation tryout

### Preprocessing inputs

In [36]:
# Preprocessing (remove stop words)
# user_input = "My dog has been vomiting has Limb Swelling and has diarrhea"
user_input = "My dog has been vomiting and has diarrhea "

# Tokenize the sentence
tokens = nltk.word_tokenize(user_input)

# Remove stop words, (stopwords are getting fromt the NLTK library)
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

# Filtered tokens as sentence
filtered_user_input = " ".join(filtered_tokens)

In [37]:
filtered_user_input

'dog vomiting diarrhea'

In [38]:
from nltk import word_tokenize

def pos_tag(sentence):
    tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

annotated_entites = [en for en in list(
      pos_tag(word_tokenize(filtered_user_input)) # Filter entities annotate as "SYMPTOM"
    ) if en[1] == "SYMPTOM"]

In [39]:
annotated_entites

[('vomiting', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')]

### Processing entity linker component

In [50]:
link_entity = []

for entity in annotated_entites:
  current_entity_candidates = get_most_similarity_entities(entity[0], expanded_symp)
  link_entity.append(get_highest_ctx_similarity(entity[0], current_entity_candidates))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 62ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 58ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

1/1 [==============================] - 0s 60ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 58ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 64ms/step


In [51]:
link_entity

[[('https://ontology.drpawspaw.com/ADSO0000000008',
   'Vomiting',
   0.9984850287437439)],
 [('https://ontology.drpawspaw.com/ADSO0000000009',
   'Diarrhea',
   0.9998852014541626)]]

### Query symptoms to get the disease

In [52]:
entities = list(map(lambda x: x[0][0], link_entity))

In [53]:
entities

['https://ontology.drpawspaw.com/ADSO0000000008',
 'https://ontology.drpawspaw.com/ADSO0000000009']

In [54]:
curr_query = build_query(entities)

In [55]:
curr_query

'\n    PREFIX adso: <https://ontology.drpawspaw.com/>\n    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n    SELECT ?diseaseName\n    WHERE {\n        ?diseaseUri adso:hasSymptom adso:ADSO0000000008 .\n?diseaseUri adso:hasSymptom adso:ADSO0000000009 .\n        ?diseaseUri adso:text ?diseaseName .\n    }\n    '

In [56]:
for rq in g.query(curr_query):
  print(rq)

(rdflib.term.Literal('Babesiosis', lang='en'),)
(rdflib.term.Literal('Distempter', lang='en'),)


In [57]:
WORD_JOIN_CHAR = " "; # Use to join the words in an array

from nltk import word_tokenize

def pos_tag(sentence):
    tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

def entity_linker(sentence):
  etq = [] # Entities to query
  annotated_entites = [en for en in list(
      pos_tag(word_tokenize(filtered_user_input)) # Filter entities annotate as "SYMPTOM"
    ) if en[1] == "SYMPTOM"]
  print("Identified named entities => ", annotated_entites)
  for e in annotated_entites:
    current_entity_candidates = get_most_similarity_entities(e[0], expanded_symp)
    print("Candidates => ",e, current_entity_candidates)
    etq.append(get_highest_ctx_similarity(e, current_entity_candidates))
    print("Named Entity => ",e, etq)
  current_query = build_query(list(map(lambda x: x[0][0], etq)))
  return [rq for rq in g.query(current_query)]

In [58]:
entity_linker("My dog has been vomiting has Limb Swelling and has diarrhea")

Identified named entities =>  [('vomiting', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')]
Candidates =>  ('vomiting', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', tensor([[1.0000]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000057', 'Nausea', tensor([[0.7699]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', tensor([[0.6183]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000066', 'Eating Poison', tensor([[0.5464]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000007', 'coughing', tensor([[0.5324]], device='cuda:0'))]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 61ms/step


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 91ms/step


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 108ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 100ms/step
Named Entity =>  ('diarrhea', 'SYMPTOM') [[('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', 0.9960421323776245)], [('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', 0.9990774393081665)]]


[(rdflib.term.Literal('Babesiosis', lang='en')),
 (rdflib.term.Literal('Distempter', lang='en'))]

In [101]:
from itertools import chain

def make_prediction(sentence):
  prediction = entity_linker(sentence)
  # prediction = [(rdflib.term.Literal('Babesiosis', lang='en'),), (rdflib.term.Literal('Distempter', lang='en'),)]
  # If there are multiple predictions from the given symptoms
  if len(prediction) > 1:
    # Retrieve the URI for above prediction
    print("prediction", prediction)
    duris = list(map(lambda x: get_uri_from_text(x[0].toPython()).toPython(), prediction))
    # Get the all unique symptoms for predicted diseases
    symptoms_uris = list(map(lambda x: get_symptoms_from_disease_uri(URIRef(x)), list(duris)))
    print("symptoms_uris", symptoms_uris)
    # Convert back to symptoms into readable format
    symptom_suggestions = list(map(lambda x: get_text_from_uri(URIRef(x)).toPython(), list(chain.from_iterable(symptoms_uris))))
    print("symptom_suggestions", symptom_suggestions)
    # Return symptoms suggestions for the better prediction
    return {
      "request": sentence,
      "result_type": "SUGGESTION",
      "symptom_suggestions": set(symptom_suggestions),
      "predicted_disease": None
    }
  # Unable to predict
  if len(prediction) < 1:
    return {
      "request": sentence,
      "result_type": "LIMITATION",
      "symptom_suggestions": None,
      "predicted_disease": None
    }
  # Return the prediction
  result = {
      "request": sentence,
      "result_type": "PREDICTION",
      "symptom_suggestions": None,
      "predicted_disease": prediction[0]
  }
  return result

In [102]:
import time

start_time = time.time()
print(make_prediction('My dog has been vomiting has Limb Swelling and has diarrhea'))
print("--- Time elaped for single execution: ", time.time()-start_time, " ---")

Identified named entities =>  [('vomiting', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')]
Candidates =>  ('vomiting', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', tensor([[1.0000]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000057', 'Nausea', tensor([[0.7699]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', tensor([[0.6183]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000066', 'Eating Poison', tensor([[0.5464]], device='cuda:0')), ('https://ontology.drpawspaw.com/ADSO0000000007', 'coughing', tensor([[0.5324]], device='cuda:0'))]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 56ms/step


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 62ms/step


...

1/1 [==============================] - 0s 60ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 59ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 57ms/step
Named Entity =>  ('diarrhea', 'SYMPTOM') [[('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', 0.9960421323776245)], [('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', 0.9990774393081665)]]
prediction [(rdflib.term.Literal('Babesiosis', lang='en'),), (rdflib.term.Literal('Distempter', lang='en'),)]
symptoms_uris [['https://ontology.drpawspaw.com/ADSO0000000008', 'https://ontology.drpawspaw.com/ADSO0000000015', 'https://ontology.drpawspaw.com/ADSO0000000005', 'https://ontology.drpawspaw.com/ADSO0000000014', 'https://ontology.drpawspaw.com/ADSO0000000009', 'https://ontology.drpawspaw.com/ADSO0000000007', 'https://ontology.drpawspaw.com/ADSO0000000006'], ['https://ontology.drpawspaw.com/ADSO0000000027', 'https://ontology.drpawspaw.com/ADSO0000000034', 'https://ontology.drpawspaw.com/ADSO0000000031', 'https://ontology.drpawspaw.com/ADSO0000000033', 'https://ontology.drpawspaw.com/ADSO0000000036', 'https://ontology.dr

## Evaluation

In [ ]:
symptoms[:5]

['Fast Breathing', 'Nausea', 'Skin Scaling', 'Severe Weight Loss', 'Lethargy']